<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1


    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance


    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-25 15:01:22
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.38 K
Current:  1.39 C
-------------------
Today PnL: -90.52 K (-0.65%)
Current PnL: -20.81 L (-14.0%)
CY Booked + Current PnL: -9.37 L (-6.3%)
-------------------
Total profit:  1.27 L
Total loss:  -22.08 L
-------------------
Total Booked + Current PnL: 17.61 L (14.47%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.24%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 86.68 L (62.46%)
Deployed:  1.22 C
Current:  1.39 C
CAGR/XIRR %: 8.0%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,12.39,56.0,H-MC,3.84,193900.0,29230.0,14097.0,-0.27,17.75,7.27,26.31,89.0,2.07,1.40,27.15,X40N,NTT,AC
77,TTKPRESTIG,770.00,98.51,45.0,M-SC,2.90,85523.0,-15254.0,15343.0,0.84,-15.14,17.94,0.09,245.0,-0.99,0.62,12.36,OX40N,NTT,DURABLES
50,NATIONALUM,244.55,-45.32,55.0,H-MC,10.45,109238.0,8734.0,20373.0,0.01,8.69,18.65,28.96,79.0,0.43,0.79,45.61,MH,ATH,METALS
84,VOLTAS,1530.00,-1.50,48.0,H-MC,3.07,206700.0,14958.0,22799.0,0.42,7.80,11.03,19.69,99.0,0.66,1.49,15.13,XY25,NTT,AC
57,RELIANCE,1533.00,-14.41,41.0,H-LC,4.29,213985.0,3619.0,25165.0,-0.82,1.72,11.76,13.68,37.0,0.14,1.54,18.53,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,485.00,-65.02,51.0,H-MC,3.22,248968.0,-52783.0,214212.0,3.12,-17.49,86.04,53.50,116.0,-0.25,1.79,10.58,XY24,NTT,FMCG
32,HINDZINC,730.22,24.68,63.0,M-LC,2.47,203145.0,-1931.0,114493.0,3.11,-0.94,56.36,54.89,52.0,-0.02,1.46,22.93,X5K,ATH,METALS
61,SAMMAANCAP,326.00,-171.59,58.0,M-SC,5.51,83028.0,-19182.0,112569.0,2.28,-18.77,135.58,91.37,208.0,-0.17,0.60,34.78,XY25,NTT,FINANCE
66,SIS,528.00,2102.59,44.0,H-SC,4.46,88880.0,-22152.0,45755.0,2.00,-19.95,51.48,21.26,156.0,-0.48,0.64,20.09,OX40N,NTT,MISC
46,KPIGREEN,731.05,-0.19,39.0,H-SC,9.18,119083.0,-6214.0,65138.0,1.58,-4.96,54.70,47.03,141.0,-0.10,0.86,49.10,MH,ATH,POWER


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,46.0,L-SC,42.74,81350.0,-32199.0,72247.0,-5.02,-28.36,88.81,35.27,268.0,-0.45,0.59,99.11,XR,NTT,HOTELS
71,TANLA,1963.11,-17.45,62.0,H-SC,16.28,221253.0,-49508.0,379449.0,-3.45,-18.28,171.50,121.86,133.0,-0.13,1.59,73.08,AR,ATH,IT
85,WHIRLPOOL,2270.00,-44.33,33.0,M-SC,1.48,94080.0,2582.0,76167.0,-3.21,2.82,80.96,86.07,223.0,0.03,0.68,36.62,XR,NTT,DURABLES
7,ATULAUTO,844.00,3624.32,50.0,M-SC,6.54,116077.0,-31291.0,83111.0,-3.12,-21.23,71.60,35.16,236.0,-0.38,0.84,18.56,XY24,NTT,AUTO
73,TATAMOTORS,1065.00,-23.48,37.0,M-LC,2.28,240796.0,-35727.0,145802.0,-2.87,-12.92,60.55,39.81,54.0,-0.25,1.74,15.40,XY24,NTT,AUTO


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,ICICIPRULI,790.00,-21.68,41.0,H-MC,2.13,134786.0,-1002.0,43752.0,-0.04,-0.74,32.46,31.48,107.0,-0.02,0.97,11.29,X40,ATH,INSURANCE
23,DABUR,735.00,-8.70,36.0,H-MC,6.79,247010.0,-1058.0,109475.0,-1.64,-0.43,44.32,43.70,102.0,-0.01,1.78,13.96,XY24,BTT,FMCG
86,WIPRO,420.00,-14.70,38.0,M-LC,5.50,149990.0,-955.0,110408.0,-1.09,-0.63,73.61,72.51,53.0,-0.01,1.08,5.35,XR,NTT,IT
32,HINDZINC,730.22,24.68,63.0,M-LC,2.47,203145.0,-1931.0,114493.0,3.11,-0.94,56.36,54.89,52.0,-0.02,1.46,22.93,X5K,ATH,METALS


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,92.53,50.0,H-SC,9.92,121500.0,-20979.0,40496.0,-0.02,-14.72,33.33,13.70,163.0,-0.52,0.88,43.40,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,98.51,45.0,M-SC,2.90,85523.0,-15254.0,15343.0,0.84,-15.14,17.94,0.09,245.0,-0.99,0.62,12.36,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,50.0,H-SC,1.27,220689.0,-48978.0,85318.0,-0.03,-18.16,38.66,13.47,138.0,-0.57,1.59,11.62,XY24,NTT,PAINTS
18,CERA,9475.0,-22.67,35.0,H-SC,1.99,141082.0,-34821.0,76847.0,0.19,-19.80,54.47,23.89,149.0,-0.45,1.02,21.84,OX40N,NTT,CERAMICS
66,SIS,528.0,2102.59,44.0,H-SC,4.46,88880.0,-22152.0,45755.0,2.00,-19.95,51.48,21.26,156.0,-0.48,0.64,20.09,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,49.96,69.0,M-MC,10.54,236052.0,11090.0,156573.0,0.54,4.93,66.33,74.53,192.0,0.07,1.70,38.77,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,12.39,56.0,H-MC,3.84,193900.0,29230.0,14097.0,-0.27,17.75,7.27,26.31,89.0,2.07,1.40,27.15,X40N,NTT,AC
84,VOLTAS,1530.00,-1.50,48.0,H-MC,3.07,206700.0,14958.0,22799.0,0.42,7.80,11.03,19.69,99.0,0.66,1.49,15.13,XY25,NTT,AC
17,CAMS,5211.76,-4.50,48.0,H-SC,1.85,108690.0,6686.0,37237.0,-0.03,6.55,34.26,43.06,122.0,0.18,0.78,24.79,X40N,ATH,MISC
79,UNIONBANK,163.00,-11.63,63.0,M-LC,6.93,158725.0,17885.0,26602.0,0.31,12.70,16.76,31.59,66.0,0.67,1.14,42.55,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-45.32,55.0,H-MC,10.45,109238.0,8734.0,20373.0,0.01,8.69,18.65,28.96,79.0,0.43,0.79,45.61,MH,ATH,METALS
11,BANKINDIA,190.00,-27.95,64.0,H-MC,7.88,185114.0,5306.0,107107.0,-0.52,2.95,57.86,62.52,88.0,0.05,1.33,38.04,XR,NTT,BANKS
85,WHIRLPOOL,2270.00,-44.33,33.0,M-SC,1.48,94080.0,2582.0,76167.0,-3.21,2.82,80.96,86.07,223.0,0.03,0.68,36.62,XR,NTT,DURABLES
37,INDIAMART,4850.92,-54.83,27.0,H-SC,10.47,126045.0,2709.0,131049.0,0.06,2.20,103.97,108.45,119.0,0.02,0.91,25.04,AR,ATH,MISC
86,WIPRO,420.00,-14.70,38.0,M-LC,5.50,149990.0,-955.0,110408.0,-1.09,-0.63,73.61,72.51,53.0,-0.01,1.08,5.35,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-19.89,32.0,H-LC,7.99,288472.0,-27370.0,140024.0,-0.10,-8.67,48.54,35.67,5.0,-0.20,2.08,3.99,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-16.33,33.0,H-LC,7.36,211121.0,-40647.0,93864.0,-2.36,-16.14,44.46,21.13,27.0,-0.43,1.52,13.56,X40,ATH,PAINTS
74,TCS,4476.75,-30.29,33.0,H-LC,12.08,278071.0,-67885.0,142734.0,-2.54,-19.62,51.33,21.64,1.0,-0.48,2.00,0.00,X40,ATH,IT
44,JIOFIN,387.00,-4.31,33.0,H-LC,13.11,259978.0,-7387.0,72846.0,-1.45,-2.76,28.02,24.48,48.0,-0.10,1.87,50.69,XY24,BTT,FINANCE
42,ITC,452.00,-39.61,37.0,H-LC,2.10,195380.0,-4758.0,25653.0,-0.36,-2.38,13.13,10.44,4.0,-0.19,1.41,3.05,X40,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-5.78,45.0,H-LC,1.65,233595.0,-12556.0,91756.0,-0.31,-5.10,39.28,32.18,16.0,-0.14,1.68,29.10,X200,ATH,IT
42,ITC,452.0,-39.61,37.0,H-LC,2.10,195380.0,-4758.0,25653.0,-0.36,-2.38,13.13,10.44,4.0,-0.19,1.41,3.05,X40,NTT,FMCG
65,SIEMENS,4671.5,-1.76,46.0,H-LC,2.83,158000.0,-28095.0,75571.0,-1.28,-15.10,47.83,25.51,15.0,-0.37,1.14,16.43,AR,ATH,ELECTRICAL
51,NESTLEIND,1377.0,-10.64,46.0,H-LC,4.22,275020.0,9594.0,47193.0,-0.28,3.61,17.16,21.40,11.0,0.20,1.98,10.50,XY25,NTT,FMCG
57,RELIANCE,1533.0,-14.41,41.0,H-LC,4.29,213985.0,3619.0,25165.0,-0.82,1.72,11.76,13.68,37.0,0.14,1.54,18.53,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-30.29,33.0,H-LC,12.08,278071.0,-67885.0,142734.0,-2.54,-19.62,51.33,21.64,1.0,-0.48,2.00,0.00,X40,ATH,IT
42,ITC,452.00,-39.61,37.0,H-LC,2.10,195380.0,-4758.0,25653.0,-0.36,-2.38,13.13,10.44,4.0,-0.19,1.41,3.05,X40,NTT,FMCG
80,UNITDSPR,1644.00,-13.23,49.0,H-LC,7.13,227384.0,-6760.0,55391.0,-2.14,-2.89,24.36,20.77,86.0,-0.12,1.64,3.83,X40N,NTT,BREWERIES
83,VBL,671.64,-19.89,32.0,H-LC,7.99,288472.0,-27370.0,140024.0,-0.10,-8.67,48.54,35.67,5.0,-0.20,2.08,3.99,X40N,ATH,FMCG
30,HCLTECH,1943.91,-2.37,39.0,H-LC,9.88,223113.0,-18811.0,82083.0,-1.47,-7.78,36.79,26.15,8.0,-0.23,1.61,5.43,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-1.76,46.0,H-LC,2.83,158000.0,-28095.0,75571.0,-1.28,-15.10,47.83,25.51,15.0,-0.37,1.14,16.43,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.61,37.0,H-LC,2.10,195380.0,-4758.0,25653.0,-0.36,-2.38,13.13,10.44,4.0,-0.19,1.41,3.05,X40,NTT,FMCG
19,CIPLA,1795.00,-22.52,39.0,H-LC,4.87,207089.0,2589.0,38829.0,-1.72,1.27,18.75,20.25,10.0,0.07,1.49,10.11,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-16.33,33.0,H-LC,7.36,211121.0,-40647.0,93864.0,-2.36,-16.14,44.46,21.13,27.0,-0.43,1.52,13.56,X40,ATH,PAINTS
57,RELIANCE,1533.00,-14.41,41.0,H-LC,4.29,213985.0,3619.0,25165.0,-0.82,1.72,11.76,13.68,37.0,0.14,1.54,18.53,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-30.29,33.0,H-LC,12.08,278071.0,-67885.0,142734.0,-2.54,-19.62,51.33,21.64,1.0,-0.48,2.00,0.00,X40,ATH,IT
40,INFY,2275.00,-18.88,45.0,H-LC,8.09,315560.0,2672.0,169014.0,-0.88,0.85,53.56,54.87,3.0,0.02,2.27,7.56,X40,BTT,IT
42,ITC,452.00,-39.61,37.0,H-LC,2.10,195380.0,-4758.0,25653.0,-0.36,-2.38,13.13,10.44,4.0,-0.19,1.41,3.05,X40,NTT,FMCG
83,VBL,671.64,-19.89,32.0,H-LC,7.99,288472.0,-27370.0,140024.0,-0.10,-8.67,48.54,35.67,5.0,-0.20,2.08,3.99,X40N,ATH,FMCG
1,ABB,7934.00,-40.16,46.0,H-LC,8.30,250114.0,-11505.0,130710.0,-0.51,-4.40,52.26,45.57,7.0,-0.09,1.80,6.43,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,46.0,L-SC,42.74,81350.0,-32199.0,72247.0,-5.02,-28.36,88.81,35.27,268.0,-0.45,0.59,99.11,XR,NTT,HOTELS
61,SAMMAANCAP,326.00,-171.59,58.0,M-SC,5.51,83028.0,-19182.0,112569.0,2.28,-18.77,135.58,91.37,208.0,-0.17,0.60,34.78,XY25,NTT,FINANCE
3,ALKYLAMINE,4546.37,-7.53,39.0,H-SC,11.43,86090.0,-14873.0,109403.0,-1.06,-14.73,127.08,93.63,148.0,-0.14,0.62,28.23,SR,ATH,CHEMICALS
49,MASFIN,398.61,-17.34,48.0,H-SC,12.66,93690.0,-4290.0,25896.0,-1.22,-4.38,27.64,22.05,152.0,-0.17,0.68,36.19,XR,ATH,FINANCE
50,NATIONALUM,244.55,-45.32,55.0,H-MC,10.45,109238.0,8734.0,20373.0,0.01,8.69,18.65,28.96,79.0,0.43,0.79,45.61,MH,ATH,METALS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1182.30,66.0,M-SC,11.35,174317.0,-6826.0,52295.0,-2.72,-3.77,30.00,25.10,235.0,-0.13,1.26,39.70,XY24,NTT,HEALTHCARE
71,TANLA,1963.11,-17.45,62.0,H-SC,16.28,221253.0,-49508.0,379449.0,-3.45,-18.28,171.50,121.86,133.0,-0.13,1.59,73.08,AR,ATH,IT
61,SAMMAANCAP,326.00,-171.59,58.0,M-SC,5.51,83028.0,-19182.0,112569.0,2.28,-18.77,135.58,91.37,208.0,-0.17,0.60,34.78,XY25,NTT,FINANCE
60,SAIL,228.00,49.96,69.0,M-MC,10.54,236052.0,11090.0,156573.0,0.54,4.93,66.33,74.53,192.0,0.07,1.70,38.77,XY24,BTT,STEEL
81,VAIBHAVGBL,521.00,57.02,55.0,H-SC,5.97,135913.0,-46862.0,157931.0,-1.76,-25.64,116.20,60.77,125.0,-0.30,0.98,21.73,XR,NTT,JEWELLERY


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.19
1,25,43.64
2,50,74.81


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.94
LC    32.33
MC    23.71
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.31
X40      14.71
X40N     11.39
XR       11.20
XY25     10.13
AR        8.34
OX40N     7.89
X200      1.68
MH        1.65
X5K       1.46
SR        1.22
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.40
H-LC    25.86
H-MC    20.73
M-SC    13.05
M-LC     5.42
M-MC     2.67
L-SC     1.49
L-LC     1.05
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.00,-5.62,40.48
IT,12.89,-17.39,81.39
FINANCE,10.06,-17.51,68.78
MISC,7.00,-18.28,84.03
BANKS,6.17,-16.57,77.89
PAINTS,5.65,-17.80,35.37
ELECTRICAL,5.46,-8.52,46.59
INSURANCE,3.76,-5.87,45.45
AC,3.59,2.71,13.58


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3175758.0,22
XR,1357561.0,14
AR,1175076.0,9
X40,805268.0,10
X40N,624478.0,9
OX40N,552312.0,10
XY25,433975.0,7
SR,251783.0,2
X5K,114493.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3713900.0,29
M-SC,1437431.0,17
H-MC,1229428.0,15
H-LC,1229254.0,15
M-LC,397305.0,4
M-MC,306587.0,2
L-SC,254946.0,3
L-MC,60257.0,1
L-LC,38863.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1239310.0      6
M-SC       XY24       821564.0      7
H-SC       AR         799805.0      5
           XR         788605.0      7
H-MC       XY24       563047.0      4
H-LC       X40        513348.0      5
H-SC       X40N       309691.0      4
M-MC       XY24       306587.0      2
H-SC       OX40N      259568.0      4
           SR         251783.0      2
H-LC       X40N       234244.0      3
H-MC       X40        216679.0      4
H-LC       AR         206281.0      2
M-LC       XY24       172404.0      2
H-MC       XY25       171764.0      2
M-SC       AR         168990.0      2
L-SC       XR         168286.0      2
M-SC       OX40N      136169.0      4
           XR         122898.0      2
M-LC       X5K        114493.0      1
M-SC       XY25       112569.0      1
H-LC       XY25       110779.0      3
M-LC       XR         110408.0      1
H-MC       XR         107107.0      1
H-LC       X200        91756.0      1
L-SC       OX40N       86660.0      1
H-MC       X40N        80543.0      2
M-SC       X40         75241.0      1
H-LC       XY24        72846.0      1
H-MC       OX40N       69915.0      1
H-SC       MH          65138.0      1
L-MC       XR          60257.0      1
L-LC       XY25        38863.0      1
H-MC       MH          20373.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 48.0 seconds
